In [1]:
# base on https://huggingface.co/docs/transformers/training

In [2]:
!rm -rf ojt_bert
!git clone https://github.com/wzwzeyal/ojt_bert

Cloning into 'ojt_bert'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 227 (delta 101), reused 182 (delta 59), pack-reused 0
Receiving objects: 100% (227/227), 9.30 MiB | 38.56 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [3]:
!pip install datasets transformers

     |████████████████████████████████| 311 kB 5.1 MB/s 
     |████████████████████████████████| 3.4 MB 58.2 MB/s 
     |████████████████████████████████| 133 kB 65.2 MB/s 
     |████████████████████████████████| 243 kB 59.8 MB/s 
     |████████████████████████████████| 1.1 MB 57.0 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 70.5 MB/s 
     |████████████████████████████████| 895 kB 58.8 MB/s 
     |████████████████████████████████| 3.3 MB 47.7 MB/s 
     |████████████████████████████████| 144 kB 70.8 MB/s 
     |████████████████████████████████| 271 kB 73.9 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from datasets import load_dataset, DatasetDict, Dataset



# Configuration

In [5]:
MODEL_CKPT = "onlplab/alephbert-base"
TEXT_COLUMN_NAME = "comment"
LABEL_COLUMN_NAME = "label"
HUGGINGFACE_LABEL_COLUMN_NAME = "labels"
NUM_LABELS = 3
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
NUM_EPOCS = 1

# Preparing the datasets


In [6]:
import pandas as pd

train_df = pd.read_csv('./ojt_bert/data/for_sentiment/train_token_df.gz').head(128)
test_df = pd.read_csv('./ojt_bert/data/for_sentiment/val_token_df.gz').head(32)
raw_datasets = DatasetDict()
raw_datasets["train"] = Dataset.from_pandas(train_df[[TEXT_COLUMN_NAME, LABEL_COLUMN_NAME]])
raw_datasets["test"] = Dataset.from_pandas(test_df[[TEXT_COLUMN_NAME, LABEL_COLUMN_NAME]])
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['comment', 'label'],
        num_rows: 128
    })
    test: Dataset({
        features: ['comment', 'label'],
        num_rows: 32
    })
})

In [7]:
# raw_datasets = load_dataset("imdb")

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    return tokenizer(examples[TEXT_COLUMN_NAME], padding="max_length", truncation=True)

In [10]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 128
    })
    test: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32
    })
})

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_CKPT, num_labels=NUM_LABELS)

Downloading:   0%|          | 0.00/481M [00:00<?, ?B/s]

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [13]:
from transformers import Trainer


trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["test"]
)

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: comment.
***** Running training *****
  Num examples = 128
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 48


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=48, training_loss=0.2678786913553874, metrics={'train_runtime': 34.6882, 'train_samples_per_second': 11.07, 'train_steps_per_second': 1.384, 'total_flos': 101035552407552.0, 'train_loss': 0.2678786913553874, 'epoch': 3.0})

In [15]:
break;

SyntaxError: ignored

In [16]:
preciations = trainer.predict(tokenized_datasets["test"])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: comment.
***** Running Prediction *****
  Num examples = 32
  Batch size = 8


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples[TEXT_COLUMN_NAME], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns([TEXT_COLUMN_NAME])
tokenized_datasets = tokenized_datasets.rename_column(LABEL_COLUMN_NAME, HUGGINGFACE_LABEL_COLUMN_NAME)
tokenized_datasets.set_format("torch")

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=TRAIN_BATCH_SIZE)
eval_dataloader = DataLoader(eval_dataset, batch_size=EVAL_BATCH_SIZE)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_CKPT, num_labels=NUM_LABELS)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = NUM_EPOCS
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
from tqdm.auto import tqdm

import numpy as np
from datasets import load_metric

metric= load_metric("accuracy", "f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

progress_bar = tqdm(range(num_training_steps))


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)
  
  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch[HUGGINGFACE_LABEL_COLUMN_NAME])

  metric.compute()

In [ ]:
  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch[HUGGINGFACE_LABEL_COLUMN_NAME])

  metric.compute()